# Classifying Celebrity Faces

For this experiment we use the 10 identities of the PubFig dataset: <br>
https://www.cs.columbia.edu/CAVE/databases/pubfig/ <br>
We roughly follow the structure of this
tutorial:<br>
https://machinelearningmastery.com/how-to-develop-a-face-recognition-system-using-facenet-in-keras-and-an-svm-classifier/ <br>
And use the FaceNet architecture from this repository: <br>
https://github.com/timesler/facenet-pytorch <br>
This architecture was introduced in: <br>
https://arxiv.org/pdf/1503.03832.pdf <br>
We use the MTCNN to crop input images so that they only depict the face. We then feed this into the InceptionResnet to get 
embeddings for each face. Lastly, we classify those embeddings with a simple NN.

In [1]:
import os
path = os.getcwd()
print(path)
os.chdir('/home/jupyter/src')
path = os.getcwd()
print(path)

import torch
from matplotlib import pyplot as plt
import numpy as np
from classifiers.FaceNet.Facenet import crop_images_batch, encode_pubfig
from facenet_pytorch import MTCNN, InceptionResnetV1

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))


/home/jupyter/src/classifiers/FaceNet
/home/jupyter/src
Running on device: cpu


Load dataset, crop images, get embeddings and save them:

In [ ]:
crop, labels = crop_images_batch(device, '/home/jupyter/src/data/pubfig83', postprocess=True)
np.savez_compressed('/home/jupyter/src/data/pubfig83/pubfig83_crop.npz', crop=crop, labels=labels)


In [2]:
#import random
#crop = np.load('/home/jupyter/src/data/pubfig83/pubfig83_crop.npz')['crop']
#print(crop.shape)
#y_train = np.load('/home/jupyter/src/data/pubfig83/pubfig83_crop.npz')['labels']
#test = []
#test_labels = []
#train = []
#train_labels = []
#random.seed(a=0)
#for id, elem in enumerate(crop):
#    rand = random.random()
#    if rand > 0.95:
#        test.append(elem)
#        test_labels.append(y_train[id])
#    else:
#        train.append(elem)
#        train_labels.append(y_train[id])
#np.savez_compressed('/home/jupyter/src/data/pubfig83/pubfig83_crop_test.npz', data=test, labels=test_labels)
#np.savez_compressed('/home/jupyter/src/data/pubfig83/pubfig83_crop_train.npz', data=train, labels=train_labels)

(13801, 3, 160, 160)


In [8]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
embeddings = []
for image in train_crop:
    embedding = resnet(torch.Tensor(np.expand_dims(image, axis=0))).detach().numpy()
    embeddings.append(embedding)
y_train = np.load('/home/jupyter/src/data/pubfig83/pubfig83_crop_train.npz')['labels']
np.savez_compressed('/home/jupyter/src/data/pubfig83/pubfig83_embedding.npz', X_train=embeddings, y_train=y_train)

Read in all embeddings and labels and normalize them to an L2 norm:

In [ ]:
from os import walk
import os
path = '../../../data/PubFig/embedding'
embeddings = None
labels = None
for (dirpath, dirnames, filenames) in walk(path):
    for f in filenames:
        read_em = np.load(os.path.join(path, f))['X_train']
        read_label = np.load(os.path.join(path, f))['y_train']
        if embeddings is None and labels is None:
            embeddings = read_em
            labels = read_label
        else:
            embeddings = np.concatenate((embeddings, read_em), axis=0)
            labels = np.concatenate((labels, read_label))
print(embeddings.shape)
print(labels.shape)
embeddings = torch.nn.functional.normalize(torch.Tensor(embeddings), p=2, dim=1, eps=1e-12, out=None)

In [2]:
path = 'data/PubFig/embedding'
embeddings_old = np.load('/home/jupyter/src/data/pubfig83/pubfig83_embedding.npz')['X_train']
embeddings = []
for em in embeddings_old:
    embeddings.append(em[0])
labels = np.load('/home/jupyter/src/data/pubfig83/pubfig83_embedding.npz')['y_train']
print(len(embeddings))
print(len(embeddings[0]))
print(len(labels))
embeddings = torch.nn.functional.normalize(torch.Tensor(embeddings), p=2, dim=1, eps=1e-12, out=None)

13152
512
13152


Fit the 1-Layer NN to the data:

In [3]:
from classifiers.FaceNet.Facenet import FaceNet
classifier = FaceNet(num_classes=83, load_model=False)
loss_histor = classifier.train(torch.Tensor(embeddings), torch.Tensor(labels), num_steps=100, learning_rate=0.1)

Then we load the test images

and test the network on them

In [ ]:
from PIL import Image
_img = np.array(Image.open('../../../data/testimg/Aaron-Eckhart/images.jpg'))

plt.imshow(_img)
from facenet_pytorch import MTCNN, InceptionResnetV1

resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)
x_aligned, prob = mtcnn(_img, return_prob=True)
plt.imshow(np.swapaxes(np.swapaxes(x_aligned, 0, 2), 0, 1))

output = classifier.predict(x_aligned.unsqueeze(0))
_, orig_label = torch.max(output, dim=1)
print('prediction: ', orig_label)

Load test images that we collected ourselves (4 images for each identity) and evaluate
the network performance:

In [ ]:
test_crops = 
test_labels = 

In [ ]:
crop = np.load('/home/jupyter/src/data/pubfig83/pubfig83_crop_test.npz')['data']
labels =  np.load('/home/jupyter/src/data/pubfig83/pubfig83_crop_test.npz')['labels']
crops_normalized = []
for image in crop:
    normalized_image = (image - 0.5) / 0.50196078
    crops_normalized.append(normalized_image) 

output = []
for id, crop_image in enumerate(crops_normalized):
    print(id)
    output.append(classifier.predict(torch.Tensor(crop_image).unsqueeze(0)))
_, pred = torch.max(output, dim=1)

correct = 100*(pred == labels).sum()/len(labels)
print('Accuracy in %: ', correct.item())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
